# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [4]:
# import libraries
import re
import pandas as pd
import pickle
from sqlalchemy import create_engine
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [5]:
import nltk
nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
# load data from database
engine = create_engine('sqlite:///etl_pipelinev2.db')
df = pd.read_sql_table('ETL_Pipelinev2', engine)
df = df.dropna()
df = df[df['related'] != 2]
X = df.message
Y = df.iloc[:, 4:].astype('int64')

### 2. Write a tokenization function to process your text data

In [7]:
def tokenize(text):
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    
    return clean_tokens


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', RandomForestClassifier())
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

# train classifier
pipeline.fit(X_train, y_train)

# predict on test data
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
target_names = df.columns[4:]

In [11]:
print(classification_report(y_test, y_pred, target_names=target_names))

                        precision    recall  f1-score   support

               related       0.67      0.80      0.73      1674
               request       0.43      0.17      0.24       884
                 offer       0.00      0.00      0.00         2
           aid_related       0.43      0.20      0.27       978
          medical_help       0.08      0.01      0.01       140
      medical_products       0.00      0.00      0.00        80
     search_and_rescue       0.00      0.00      0.00        58
              security       0.00      0.00      0.00        29
              military       0.00      0.00      0.00        14
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00       199
                  food       0.23      0.02      0.03       386
               shelter       0.07      0.00      0.01       255
              clothing       0.00      0.00      0.00        24
                 money       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [113]:
pipeline.get_params()

AttributeError: 'DataFrame' object has no attribute 'get_params'

In [128]:
parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
    'clf__n_estimators': ([50, 100, 200])
    
}

cv = GridSearchCV(pipeline, param_grid=parameters) 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [129]:
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__ngram_range': ((1, 1), (1, 2)), 'clf__n_estimators': [50, 100, 200]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [130]:
y_pred = cv.predict(X_test)

In [131]:
print(classification_report(y_test, y_pred, target_names=target_names))

                        precision    recall  f1-score   support

               related       0.70      0.80      0.74      1718
               request       0.48      0.05      0.09       928
                 offer       0.00      0.00      0.00         4
           aid_related       0.50      0.09      0.15      1016
          medical_help       0.00      0.00      0.00       158
      medical_products       0.00      0.00      0.00        91
     search_and_rescue       0.00      0.00      0.00        57
              security       0.00      0.00      0.00        33
              military       0.00      0.00      0.00        11
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00       202
                  food       1.00      0.00      0.00       403
               shelter       0.50      0.00      0.01       287
              clothing       0.00      0.00      0.00        27
                 money       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [133]:
pickle.dump(cv, open('cv.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.